In [ ]:
# Imports
from IPython.display import display, HTML
from pymongo import MongoClient
import os
from src.repository import EmbeddingsRepository, COLLECTION, MONGO_URI_KEY
import gensim.downloader as api
from src.embeddings import get_text_embedding
import numpy as np


In [ ]:
# Inits
uri = os.environ.get(MONGO_URI_KEY)
client = MongoClient(uri)
db = client.get_default_database()
repository = EmbeddingsRepository()
gensim_model = api.load('glove-wiki-gigaword-50')

In [ ]:
# Helpers
def display_items(items: list, cap: int = 4) -> None:
    items = items[:cap]
    html_code = '''
    <style>
        .container {
            display: flex;
            flex-wrap: wrap;
            gap: 14px; /* Further increased gap */
            justify-content: flex-start; /* Left-align the items */
        }
        .card {
            max-width: 240px; /* Further increased card width */
            box-shadow: 0 3px 6px rgba(0, 0, 0, 0.1);
            text-align: left; /* Left-align the text */
            font-family: WoltHeading-Omnes, -apple-system, BlinkMacSystemFont, Roboto, "Segoe UI", Arimo, "Open Sans", "Helvetica Neue", sans-serif;
            font-feature-settings: "kern", "ss01", "ss05", "ss07";
            -webkit-font-smoothing: antialiased;
            text-rendering: optimizelegibility;
            font-variant-ligatures: common-ligatures;
            color: #2A2922;
            background-color: #F8F5F2; /* Background color */
            border-radius: 8px;
        }
        .image-container {
            position: relative;
            width: 100%;
            height: 0;
            padding-top: 56.25%; /* 16:9 Aspect Ratio */
            overflow: hidden;
            border-top-left-radius: 8px; /* Rounded corners for the top */
            border-top-right-radius: 8px;
        }
        .image-container img {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            object-fit: cover;
            object-position: 50% 50%;
        }
        .details {
            padding: 14px; /* Further increased padding */
        }
        .details .item-name {
            font-size: 1.66rem; /* Further increased font size */
            line-height: 2rem; /* Increased line height */
            font-weight: 700;
            font-style: normal;
            font-stretch: normal;
            text-transform: none;
            color: #2A2922;
            margin: 0;
        }
        .details .venue-info {
            font-size: 0.9rem; /* Increased font size */
            line-height: 1.1rem; /* Increased line height */
            font-weight: 400;
            color: #605A4C; /* Slightly tinted color */
            margin: 2px 0; /* Almost no padding */
        }
        .details p {
            font-size: 0.9rem; /* Increased font size */
            color: #2A2922;
        }
    </style>
    <div class="container">
    '''
    
    for item in items:
        item_html = f'''
        <div class="card">
            <div class="image-container">
                <img src="{item.get("itemImageUrl", "")}" alt="Dish Image">
            </div>
            <div class="details">
                <div class="item-name">{item.get("itemName", "")}</div>
                <div class="venue-info">{item.get("venueName", "")}, {item.get("venueCity", "")}</div>
                <p>{item.get("itemDescription", "")}</p>
            </div>
        </div>
        '''
        html_code += item_html
    
    html_code += '</div>'
    
    display(HTML(html_code))

def get_random_item(city: str = None) -> dict:
    pipeline = []

    if city:
        pipeline.append({"$match": {"venueCity": city}})

    pipeline.extend([
        {"$sample": {"size": 1}},
        {"$project": {"_id": 0}}
    ])

    return list(db[COLLECTION].aggregate(pipeline))[0]

def filter_items(*, anchor_item: dict, similar_items: list[dict]) -> list[dict]:
    recommended_items = []

    for item in similar_items:
        if item["venueName"] != anchor_item["venueName"]:
            recommended_items.append(item)

    return recommended_items


In [ ]:
# Gensim
base_vector = gensim_model.get_vector("london")
vector_to_substract = gensim_model.get_vector("england")
vector_to_add = gensim_model.get_vector("japan")

result = base_vector - vector_to_substract + vector_to_add
print(gensim_model.similar_by_vector("london"))
print(gensim_model.similar_by_vector(result))

In [ ]:
# Similar Items
random_item = get_random_item()
query_vector = random_item["textEmbedding"]
similar_items = repository.search_similar_items(query_vector, 30, random_item["venueCity"])
similar_items = filter_items(anchor_item=random_item, similar_items=similar_items)
display_items([random_item])
display_items(similar_items)

In [ ]:
# Similar Images
random_item = get_random_item()
query_vector = random_item["imageEmbedding"]
similar_images = repository.search_similar_images(query_vector, 5, "Helsinki")
display_items([random_item])
display_items(similar_images)

In [ ]:
# Search
query_text = "salad"
query_vector = get_text_embedding(query_text)
similar_items = repository.search_similar_items(query_vector, 5, "Helsinki")
display_items(similar_items)

In [ ]:
# Search continued
query_text = "salad"
exclude = "tomato"
query_vector = np.array(get_text_embedding(query_text)) - np.array(get_text_embedding(exclude))
similar_items = repository.search_similar_items(list(query_vector), 5, "Helsinki")
display_items(similar_items)